In [10]:
!apt-get update
!apt-get install -y tesseract-ocr tesseract-ocr-spa poppler-utils
!pip install pytesseract pdf2image pillow PyPDF2

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-spa is already th

In [11]:
import os
import re
import shutil
import pytesseract
from pdf2image import convert_from_path
from google.colab import files
from PyPDF2 import PdfReader, PdfWriter

In [63]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

Saving image2026-02-06-160559-4.pdf to image2026-02-06-160559-4.pdf


In [64]:
pages = convert_from_path(
    pdf_path,
    dpi=300
)

In [65]:
texto_completo = ""

for page in pages:
    texto = pytesseract.image_to_string(
        page,
        lang="spa",
        config="--psm 6"
    )
    texto_completo += texto + " "

In [66]:
def extraer_nombre_cc(texto):
    texto = texto.upper().replace("\n", " ")

    patron = re.compile(
        r"([A-ZÁÉÍÓÚÑ\s]+)\s*[\(\[]\s*C\s*C\s*(\d+)\s*[\)\]]",
        re.UNICODE
    )

    match = patron.search(texto)

    if match:
        #nombre = match.group(1).strip()
        #cedula = match.group(2)
        #return nombre, cedula
        return match.group(1).strip()

    #return None, None
    return None


In [67]:
def extraer_fecha(texto):
    texto = texto.upper().replace("\n", " ")

    patron_fecha = re.compile(
        r"FECHA\s*Y\s*HORA\s*[:\-]?\s*(\d{2}/\d{2}/\d{4})",
        re.UNICODE
    )

    match = patron_fecha.search(texto)

    if match:
        return match.group(1)  # DD/MM/YYYY

    return None

In [68]:
def formatear_fecha(fecha):
    # 27/02/2026 → 2026-02-27
    d, m, y = fecha.split("/")
    return f"{y}-{m}-{d}"

In [69]:
os.makedirs("salida", exist_ok=True)

reader = PdfReader(pdf_path)

for i, (page_img, page_pdf) in enumerate(zip(pages, reader.pages), start=1):

    texto = pytesseract.image_to_string(
        page_img,
        lang="spa",
        config="--psm 6"
    )

    nombre = extraer_nombre_cc(texto)
    fecha = extraer_fecha(texto)

    if nombre and fecha:
        nombre_limpio = "_".join(nombre.split())
        fecha_limpia = formatear_fecha(fecha)
        nombre_archivo = f"{nombre_limpio}_{fecha_limpia}.pdf"
    else:
        nombre_archivo = f"SIN_DATOS_{i}.pdf"

    writer = PdfWriter()
    writer.add_page(page_pdf)

    ruta = os.path.join("salida", nombre_archivo)

    with open(ruta, "wb") as f:
        writer.write(f)

    print(f"✅ Página {i} → {nombre_archivo}")


✅ Página 1 → CAROLINA_ACOSTA_CIFUENTES_2026-02-10.pdf
✅ Página 2 → CLAUDIA_LUCIA_BARAJAS_GUARNIZO_2026-02-13.pdf
✅ Página 3 → SEBASTIAN_PATIÑO_PEREZ_2026-02-10.pdf
✅ Página 4 → JEFFERSON_PARRA_ARIZA_2026-02-12.pdf
✅ Página 5 → FREDY_ARNOLDO_BELTRAN_PRIETO_2026-02-17.pdf
✅ Página 6 → VICTOR_MANUEL_LOPEZ_RODRIGUEZ_2026-02-13.pdf
✅ Página 7 → LUZ_STELLA_GAMBA_PENAGOS_2026-02-10.pdf
✅ Página 8 → EDWAR_JAVIER_DURANGO_ERAZO_2026-02-12.pdf
✅ Página 9 → GABRIEL_ORLANDO_CELIS_PEÑA_2026-02-17.pdf
✅ Página 10 → SERGIO_AUGUSTO_PEÑUELA_PARRA_2026-02-10.pdf
✅ Página 11 → LOWEL_ANDRES_HERNANDEZ_NAVARRO_2026-02-13.pdf
✅ Página 12 → DANIEL_FRANCISCO_SMIT_DEL_RIO_2026-02-11.pdf
✅ Página 13 → LAURA_DURAN_HOYOS_2026-02-13.pdf
✅ Página 14 → ANDRES_FELIPE_RESTREPO_VARGAS_2026-02-10.pdf
✅ Página 15 → DIANA_PAOLA_ALTURO_RAMIREZ_2026-02-11.pdf
✅ Página 16 → CESAR_AUGUSTO_NUNEZ_RAMIREZ_2026-02-11.pdf
✅ Página 17 → JENIFFER_CAROLINA_ORTIZ_BASTOS_2026-02-10.pdf
✅ Página 18 → CRISTIAN_DAVID_SANCHEZ_RAMIREZ_2026-02

In [70]:
shutil.make_archive(
    "/content/documentos_por_pagina",
    "zip",
    "/content/salida"
)

files.download("/content/documentos_por_pagina.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Para borrar la carpeta con los documentos

In [62]:
import os

carpeta = "salida"

for archivo in os.listdir(carpeta):
    ruta = os.path.join(carpeta, archivo)
    if os.path.isfile(ruta):
        os.remove(ruta)

print("✅ Carpeta limpiada")

✅ Carpeta limpiada
